[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-1/agent-memory.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239417-lesson-7-agent-with-memory)

# Agent memory

## Review

Previously, we built an agent that can:

* `act` - let the model call specific tools
* `observe` - pass the tool output back to the model
* `reason` - let the model reason about the tool output to decide what to do next (e.g., call another tool or just respond directly)

![Screenshot 2024-08-21 at 12.45.32 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbab7453080e6802cd1703_agent-memory1.png)

## Goals

Now, we're going extend our agent by introducing memory.

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph

In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

OPENAI_API_KEY: ··········


We'll use [LangSmith](https://docs.smith.langchain.com/) for [tracing](https://docs.smith.langchain.com/concepts/tracing).

In [3]:
_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

LANGCHAIN_API_KEY: ··········


This follows what we did previously.

In [20]:
from langchain_openai import ChatOpenAI

def analyze_symbol(symbol: str) -> str:
    """Provide interpretations for common dream symbols based on Jungian psychology."""
    symbol_interpretations = {
        "water": "Represents the unconscious mind, emotions, and purification.",
        "flying": "Symbolizes freedom, ambition, or a desire to rise above challenges.",
        "shadow": "Represents the unacknowledged or repressed parts of the self.",
        "forest": "Symbolizes exploration of the unknown aspects of the self or life.",
        "door": "Represents opportunities, transitions, or barriers in life.",
        "light": "Symbolizes enlightenment, awareness, or revelation.",
        "snake": "Can represent transformation, healing, or potential danger.",
        "death": "Often symbolizes change, endings, or the transition to a new phase.",
        "baby": "Represents new beginnings, innocence, or potential growth.",
        "teeth falling out": "May symbolize anxiety, loss, or concerns about self-image.",
        "chase": "Indicates avoidance of issues or feeling threatened.",
        "naked in public": "Reflects vulnerability or fear of exposure.",
        "vehicle": "Represents one's journey and control over life direction.",
        "house": "Symbolizes the self; different rooms may represent different aspects of the psyche.",
        "animals": "Can represent instincts, behaviors, or emotions.",
        "fire": "Symbolizes passion, transformation, destruction, or purification.",
        "mirror": "Represents self-reflection, identity, or confronting truths.",
        "falling": "May indicate insecurities, anxieties, or a loss of control.",
        "mountain": "Symbolizes challenges, goals, or spiritual elevation.",
        "bridge": "Represents transition, connection, or overcoming obstacles.",
        "clock": "Symbolizes time constraints, urgency, or awareness of life's passing.",
        "school": "Reflects learning experiences, personal growth, or unresolved issues from the past"
        # Add more symbols as needed
    }
    return symbol_interpretations.get(symbol.lower(), "Symbol interpretation not found.")

def analyze_emotion(emotion: str) -> str:
    """Provide interpretations based on emotions felt in the dream."""
    emotion_interpretations = {
        "fear": "May indicate underlying anxieties or unresolved conflicts.",
        "joy": "Reflects satisfaction, fulfillment, or positive developments.",
        "anger": "Could symbolize repressed frustration or feelings of injustice.",
        "sadness": "May represent grief, loss, or a need for healing.",
        "confusion": "Indicates uncertainty or a lack of clarity in waking life.",
        "peace": "Suggests harmony, acceptance, or resolution of internal conflicts.",
        "anxiety": "Reflects stress, worry, or anticipation of future events.",
        "love": "Symbolizes connections, relationships, or self-acceptance."
        # Add more emotions as needed
    }
    return emotion_interpretations.get(emotion.lower(), "Emotion interpretation not found.")

def identify_archetypes(description: str) -> str:
    """Identify Jungian archetypes present in the dream based on the description."""
    archetypes = {
        "hero": "Represents the ego and the quest for identity and wholeness.",
        "shadow": "Embodies the unconscious aspects of the personality.",
        "anima": "Represents the feminine inner personality in men.",
        "animus": "Represents the masculine inner personality in women.",
        "wise old man": "Symbolizes guidance, wisdom, and insight.",
        "child": "Represents innocence, potential, and new beginnings.",
        "mother": "Embodies nurturing, fertility, and creation.",
        "trickster": "Challenges norms, creates chaos, and brings transformation."
        # Add more archetypes as needed
    }
    # Simple keyword matching for demonstration purposes
    matched_archetypes = [name for name in archetypes if name in description.lower()]
    if matched_archetypes:
        interpretations = [f"{name.capitalize()}: {archetypes[name]}" for name in matched_archetypes]
        return "\n".join(interpretations)
    else:
        return "No specific archetypes identified."

def suggest_reflection_questions() -> str:
    """Provide reflection questions to help the user gain deeper insights."""
    questions = [
        "How did the dream make you feel upon waking?",
        "Are there any situations in your waking life that relate to the dream?",
        "What personal associations do you have with the symbols in your dream?",
        "Have you experienced similar dreams before?",
        "Is there a message or lesson that the dream might be conveying?",
        "How might the dream relate to your personal goals or challenges?"
    ]
    return "\n".join(questions)



# # List of tools
# tools = [check_ip, check_domain, suggest_remediation]

# # Set up the chat model and bind the tools
# llm = ChatOpenAI(model="gpt-4")
# llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)


In [21]:
from langchain_openai import ChatOpenAI

# Initialize the chat model
llm = ChatOpenAI(model="gpt-4")

# List of tools
tools = [analyze_symbol, analyze_emotion, identify_archetypes, suggest_reflection_questions]

# Bind the tools to the language model
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)


In [23]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# System message guiding the assistant's behavior
sys_msg = SystemMessage(content=(
    "You are a dream interpretation assistant specializing in Carl Jung's analytical psychology and modern theories. "
    "You help users understand the meanings behind their dreams, symbols, and subconscious thoughts. "
    "Use thoughtful analysis and provide insights based on established psychological concepts. "
    "When appropriate, utilize available tools to enhance your interpretations."
))

def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}


In [24]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode

# Build the graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,  # Routes to "tools" if a tool call is present; else to END
)
builder.add_edge("tools", "assistant")

# Compile the graph
dream_agent_graph = builder.compile()


## Memory

Let's run our agent, as before.

In [25]:
from langgraph.checkpoint.memory import MemorySaver

# Initialize the MemorySaver checkpointer
memory = MemorySaver()

# Compile the graph with the checkpointer
dream_agent_graph_memory = builder.compile(checkpointer=memory)


In [26]:
# User-specific configurations with unique thread IDs
user_configs = {
    "user1": {"configurable": {"thread_id": "dream_thread_user1"}},
    "user2": {"configurable": {"thread_id": "dream_thread_user2"}},
    "user3": {"configurable": {"thread_id": "dream_thread_user3"}}
}


In [27]:
from langchain_core.messages import HumanMessage

# User 1's input
messages_user1 = [HumanMessage(content=(
    "I dreamed that I was climbing a mountain and reached the top. What does it mean?"
))]

# Invoke the graph for User 1
result_state_user1 = dream_agent_graph_memory.invoke({"messages": messages_user1}, user_configs["user1"])

# Print User 1's conversation
print("User 1 Conversation:")
for m in result_state_user1['messages']:
    m.pretty_print()


User 1 Conversation:
================================ Human Message =================================

I dreamed that I was climbing a mountain and reached the top. What does it mean?
================================== Ai Message ==================================
Tool Calls:
  analyze_symbol (call_baJUtIQNdk8f3qMyXp5Htkut)
 Call ID: call_baJUtIQNdk8f3qMyXp5Htkut
  Args:
    symbol: mountain
================================= Tool Message =================================
Name: analyze_symbol

Symbolizes challenges, goals, or spiritual elevation.
================================== Ai Message ==================================

Dreaming about climbing a mountain and reaching the top represents overcoming challenges and achieving your goals. It may indicate that you are making significant progress in your personal or professional life, successfully overcoming obstacles that once seemed insurmountable.

In the context of Carl Jung's analytical psychology, mountains can also symbolize spiri

In [28]:
# User 2's input
messages_user2 = [HumanMessage(content=(
    "In my dream, I was being chased by a shadow figure, and I felt scared. What could this signify?"
))]

# Invoke the graph for User 2
result_state_user2 = dream_agent_graph_memory.invoke({"messages": messages_user2}, user_configs["user2"])

# Print User 2's conversation
print("\nUser 2 Conversation:")
for m in result_state_user2['messages']:
    m.pretty_print()



User 2 Conversation:
================================ Human Message =================================

In my dream, I was being chased by a shadow figure, and I felt scared. What could this signify?
================================== Ai Message ==================================
Tool Calls:
  identify_archetypes (call_UWmHazJB7vAhcFdZ6dVHzQ5q)
 Call ID: call_UWmHazJB7vAhcFdZ6dVHzQ5q
  Args:
    description: being chased by a shadow figure
================================= Tool Message =================================
Name: identify_archetypes

Shadow: Embodies the unconscious aspects of the personality.
================================== Ai Message ==================================
Tool Calls:
  analyze_emotion (call_3zSSUkY5OZHQO0sf12vbzZgA)
 Call ID: call_3zSSUkY5OZHQO0sf12vbzZgA
  Args:
    emotion: scared
================================= Tool Message =================================
Name: analyze_emotion

Emotion interpretation not found.
================================== Ai 

In [29]:
# User 3's input
messages_user3 = [HumanMessage(content=(
    "I dreamt of water overflowing from a glass, and I felt overwhelmed."
))]

# Invoke the graph for User 3
result_state_user3 = dream_agent_graph_memory.invoke({"messages": messages_user3}, user_configs["user3"])

# Print User 3's conversation
print("\nUser 3 Conversation:")
for m in result_state_user3['messages']:
    m.pretty_print()



User 3 Conversation:
================================ Human Message =================================

I dreamt of water overflowing from a glass, and I felt overwhelmed.
================================== Ai Message ==================================
Tool Calls:
  analyze_symbol (call_ZMUraDb6GxPhiA5tXhVFeMkx)
 Call ID: call_ZMUraDb6GxPhiA5tXhVFeMkx
  Args:
    symbol: water
================================= Tool Message =================================
Name: analyze_symbol

Represents the unconscious mind, emotions, and purification.
================================== Ai Message ==================================
Tool Calls:
  analyze_symbol (call_YpOjdbMVkrn2OPYAZy6PwlTe)
 Call ID: call_YpOjdbMVkrn2OPYAZy6PwlTe
  Args:
    symbol: glass
================================= Tool Message =================================
Name: analyze_symbol

Symbol interpretation not found.
================================== Ai Message ==================================
Tool Calls:
  analyze_emotion (

In [30]:
# User 1's follow-up message
messages_user1_followup = [HumanMessage(content=(
    "Yes, I recently completed a big project at work. But now I feel unsure about what's next."
))]

# Invoke the graph again for User 1
result_state_user1 = dream_agent_graph_memory.invoke({"messages": messages_user1_followup}, user_configs["user1"])

# Print User 1's updated conversation
print("\nUser 1 Conversation (Follow-up):")
for m in result_state_user1['messages']:
    m.pretty_print()



User 1 Conversation (Follow-up):
================================ Human Message =================================

I dreamed that I was climbing a mountain and reached the top. What does it mean?
================================== Ai Message ==================================
Tool Calls:
  analyze_symbol (call_baJUtIQNdk8f3qMyXp5Htkut)
 Call ID: call_baJUtIQNdk8f3qMyXp5Htkut
  Args:
    symbol: mountain
================================= Tool Message =================================
Name: analyze_symbol

Symbolizes challenges, goals, or spiritual elevation.
================================== Ai Message ==================================

Dreaming about climbing a mountain and reaching the top represents overcoming challenges and achieving your goals. It may indicate that you are making significant progress in your personal or professional life, successfully overcoming obstacles that once seemed insurmountable.

In the context of Carl Jung's analytical psychology, mountains can also sy